In [1]:
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [2]:
import pandas as pd

# Steps for getting the neighborhood DataFrame

- Get The Wikipedia HTML
- Set the variable "neighborhood" to the first DataFrame of the HTML
- Remove the "Not assigned" Boroughs
- Group the neighborhoods by postcode
- Drop the possible duplicates
- change name for "Not assigned" neighborhoods


In [86]:
a=pd.read_html(url)
neighborhood=a[0]
neighborhood=neighborhood[neighborhood["Borough"]!= "Not assigned"]

_=neighborhood.groupby(["Postcode","Borough"])["Neighborhood"].apply(','.join).reset_index()
_.drop_duplicates()
neighborhood=_
neighborhood

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie..."
101,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam..."


In [46]:
neighborhood.shape

(103, 3)

In [82]:
a=geocoder.geocodefarm("M9W")
a.latlng

[43.7115516667161, -79.584526062013]

In [94]:
locationPostCodes={}


KeyError: 'M1B'

In [113]:
for i in neighborhood["Postcode"]:
    locationPostCodes[i]= None
    while locationPostCodes[i] == None:
        _=geocoder.geocodefarm(i)
        locationPostCodes[i]=_.latlng
        print("Filling: ",i)

print("Done")

Filling:  M4L
Done


In [114]:
locationPostCodes

{'M1B': [43.8101539611717, -79.1946029663129],
 'M1C': [43.7846717834717, -79.1589584350129],
 'M1E': [43.7662887573717, -79.1728897094129],
 'M1G': [43.7682876586717, -79.2141113281129],
 'M1H': [43.7691802978717, -79.2387695311298],
 'M1J': [43.7439384460716, -79.2313537597129],
 'M1K': [43.7258033752716, -79.2628479003129],
 'M1L': [43.7166557312716, -79.2865371701299],
 'M1M': [43.7235755920716, -79.2344512939129],
 'M1N': [43.6978836059715, -79.2587585449129],
 'M1P': [43.7598838806716, -79.2683334350129],
 'M1R': [43.7513008117716, -79.2998123168129],
 'M1S': [43.7977066040717, -79.2677078247129],
 'M1T': [43.7849998474717, -79.2993545532129],
 'M1V': [43.8178100585717, -79.2865142822129],
 'M1W': [43.8015632629717, -79.3188705444129],
 'M1X': [43.8344345092718, -79.2189102172129],
 'M2H': [43.8029403686717, -79.356063842713],
 'M2J': [43.7811393737717, -79.348510742113],
 'M2K': [28.4271793365465, 77.0547790527126],
 'M2L': [43.7575950622716, -79.380737304613],
 'M2M': [43.79217

In [121]:
_=pd.DataFrame.from_dict(locationPostCodes,orient="index", columns=["Latitude","Longitude"]).reset_index().rename(columns={"index":"Postcode"})
latsDF=_
latsDF

,Postcode,Latitude,Longitude
0,M1B,43.810154,-79.194603
1,M1C,43.784672,-79.158958
2,M1E,43.766289,-79.172890
3,M1G,43.768288,-79.214111
4,M1H,43.769180,-79.238770
...,...,...,...
98,M9N,43.705570,-79.518616
99,M9P,43.696484,-79.529266
100,M9R,43.684505,-79.557915
101,M9V,43.742935,-79.584831


In [130]:
hoodWithLocs=neighborhood.merge(latsDF,how="inner")
hoodWithLocs

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.810154,-79.194603
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784672,-79.158958
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.766289,-79.172890
3,M1G,Scarborough,Woburn,43.768288,-79.214111
4,M1H,Scarborough,Cedarbrae,43.769180,-79.238770
...,...,...,...,...,...
98,M9N,York,Weston,43.705570,-79.518616
99,M9P,Etobicoke,Westmount,43.696484,-79.529266
100,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie...",43.684505,-79.557915
101,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",43.742935,-79.584831


In [123]:
import folium


In [145]:
torontoMap=folium.Map(zoom_start=11,location=[hoodWithLocs["Latitude"].median(),hoodWithLocs["Longitude"].median()])

for index, row in hoodWithLocs.iterrows():
    folium.Marker(
    [row["Latitude"],row["Longitude"]],
    popup=row["Postcode"]
).add_to(torontoMap)


torontoMap